In [ ]:
%pip install datasets --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.7 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

dataset_a = load_dataset("openbmb/UltraFeedback", split="train")
dataset_b = load_dataset("truthful_qa", "generation", split="validation")
dataset_c = load_dataset("truthful_qa", "multiple_choice", split="validation")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split:   0%|          | 0/817 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/817 [00:00<?, ? examples/s]

In [ ]:
dataset_a

Dataset({
    features: ['source', 'instruction', 'models', 'completions', 'correct_answers', 'incorrect_answers'],
    num_rows: 63967
})

In [ ]:
dataset_a = dataset_a.remove_columns(["models", "correct_answers", "incorrect_answers"])

In [ ]:
dataset_a = dataset_a.filter(lambda x: x["source"] != "truthful_qa")
f"Remaining samples after removing the TruthfulQA source [{dataset_a.num_rows} / 63967]"

Filter:   0%|          | 0/63967 [00:00<?, ? examples/s]

'Remaining samples after removing the TruthfulQA source [63156 / 63967]'

In [ ]:
contaminated_prompts = list(set(dataset_b["question"] + dataset_c["question"]))
dataset_a = dataset_a.filter(lambda x: x["instruction"] not in contaminated_prompts)
f"Remaining samples after removing the contaminated prompts [{dataset_a.num_rows} / 63967]"

Filter:   0%|          | 0/63156 [00:00<?, ? examples/s]

'Remaining samples after removing the contaminated prompts [63136 / 63967]'

In [ ]:
del dataset_b, dataset_c

In [ ]:
def remove_partially_annotated(dataset_row: dict) -> dict:
    completions = []
    for completion in dataset_row["completions"]:
        if not all(aspect in ["instruction_following", "honesty", "truthfulness", "helpfulness"] for aspect in completion["annotations"].keys()):
            continue
        ratings = []
        for aspect in ["instruction_following", "honesty", "truthfulness", "helpfulness"]:
            try:
                ratings.append(int(completion["annotations"][aspect]["Rating"]))
            except:
                break
        if len(ratings) != 4:
            continue
        if "overall_score" in completion:
            completion.pop("overall_score")
        if "critique" in completion:
            completion.pop("critique")
        if "custom_system_prompt" in completion:
            completion.pop("custom_system_prompt")
        if "principle" in completion:
            completion.pop("principle")
        completions.append(completion)
    dataset_row["completions"] = completions
    return dataset_row

dataset_a = dataset_a.map(remove_partially_annotated)
f"Remaining samples after removing ones partially annotated or with invalid values [{dataset_a.num_rows} / 63967]"

Map:   0%|          | 0/63136 [00:00<?, ? examples/s]

'Remaining samples after removing ones partially annotated or with invalid values [63136 / 63967]'

In [ ]:
dataset_a = dataset_a.filter(lambda x: len(x["completions"]) > 1)
f"Remaining samples after removing the rows with less than 2 annotations (at least one to be chosen and one to be rejected) [{dataset_a.num_rows} / 63967]"

Filter:   0%|          | 0/63136 [00:00<?, ? examples/s]

'Remaining samples after removing the rows with less than 2 annotations (at least one to be chosen and one to be rejected) [61478 / 63967]'

In [ ]:
def compute_rating_average(dataset_row: dict) -> dict:
    for completion in dataset_row["completions"]:
        ratings = []
        for aspect in ["instruction_following", "honesty", "truthfulness", "helpfulness"]:
            ratings.append(int(completion["annotations"][aspect]["Rating"]))
        completion.update({"average_rating": sum(ratings) / 4})
    return dataset_row

dataset_a = dataset_a.map(compute_rating_average)

Map:   0%|          | 0/61478 [00:00<?, ? examples/s]

In [ ]:
dataset_a = dataset_a.filter(lambda x: len(set(c["average_rating"] for c in x["completions"])) > 1)
f"Remaining samples after removing the ties that cannot be resolved [{dataset_a.num_rows} / 63967]"

Filter:   0%|          | 0/61478 [00:00<?, ? examples/s]

'Remaining samples after removing the ties that cannot be resolved [60917 / 63967]'

In [ ]:
from random import choice

def select_chosen_and_rejected(dataset_row: dict) -> dict:
    chosen = max(dataset_row["completions"], key=lambda x: x["average_rating"])
    rejected = choice([completion for completion in dataset_row["completions"] if completion != chosen and completion["average_rating"] < chosen["average_rating"]])
    return {
        "prompt": dataset_row["instruction"],
        "chosen": [
            {
                "role": "user",
                "content": dataset_row["instruction"],
            },
            {
                "role": "assistant",
                "content": chosen["response"],
            },
        ],
        "chosen-rating": chosen["average_rating"],
        "chosen-model": chosen["model"],
        "rejected": [
            {
                "role": "user",
                "content": dataset_row["instruction"],
            },
            {
                "role": "assistant",
                "content": rejected["response"],
            },
        ],
        "rejected-rating": rejected["average_rating"],
        "rejected-model": rejected["model"],
        "source": dataset_row["source"],
    }

dataset_a = dataset_a.map(select_chosen_and_rejected, remove_columns=dataset_a.column_names)
f"Remaining samples after computing the chosen / rejected pairs [{dataset_a.num_rows} / 63967]"

Map:   0%|          | 0/60917 [00:00<?, ? examples/s]

'Remaining samples after computing the chosen / rejected pairs [60917 / 63967]'